# Lab 05: Motion correction and slice timing

The purpose of this lab is to practice correcting for volume-to-volume motion in a functional image series, and to learn about common correction parameters and how to describe them with numbers and figures. We will also practice slice-timing correction, and test its effect on the timeseries data. 

## Preparation
* Import python tools for plotting
* Copy functional image from lab 01 into our working directory

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from nilearn import plotting
from nilearn import image

In [ ]:
%%bash

cp ../01-Lab/osfshare/sub-97_task-flanker_bold.nii.gz my_bold.nii.gz

## Describing and detecting motion with FSL tools

FSL offers a tool to detect motion outliers and to visually inspect motion and related changes in BOLD signal intensity
* [FSL Motion Outliers](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLMotionOutliers)
* This will create a dummy variable that codes for volumes possibly corrupted by motion, which can be used to ignore those volumes in our functional image analysis later. 

In [ ]:
%%bash

# Running the bare bones command like below will output only the confound matrix 
# The default metric for detecting outliers is refrms (root mean square intensity diffs with referece vol, which is middle volume)
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion 

In [ ]:
%%bash

# What do the columns represent?
cat my_bold_motion

In [ ]:
%%bash

# We can produce figures of the motion by adding the -p flag
# What is refrms?
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion -s refrms_ts -p refrms_plot

Plot of refrms:

Syntax to plot a .png in the notebook:
`![imglabel](filename.png)`

In [ ]:
%%bash

# This version of the command will use dvars as the metric for detecting outliers
# DVARS: temporal Derivatives, VARS refers to root mean square of variance of global signal timecourse
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion --dvars -s dvars_ts -p dvars_plot

Plot of dvars:

In [ ]:
%%bash

# This version of the command will use fd as the metric for detecting outliers
# FD stands for: 
fsl_motion_outliers -i my_bold.nii.gz -o my_bold_motion --fd -s fd_ts -p fd_plot

Plot of FD:

### Questions:
* What are the similarities and differences of the refrms and dvars metrics?
    * Similarities:
    * Differences: 
* How does FD differ from both refrms and dvars?
    * Which metric is "closest to the data" with respect to describing the effects of motion?

### Descriptive statistics on motion metrics

In [ ]:
dvars=pd.read_table('dvars_ts',header=None)

In [ ]:
# Preview first 5 values in timeseries
dvars.head()

In [ ]:
# Descriptives
dvars.describe()

In [ ]:
dvars.mean()

### Practice: what is the mean fd ?

In [ ]:
# Python code


## Motion correction tools in FSL

mcflirt is the basic motion correction tool provided by FSL
* [User guide](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MCFLIRT)

In [ ]:
%%bash

# Look at usage for mcflirt
mcflirt

In [ ]:
%%bash

# Example mcflirt call with defaults
# Input does not have to be skull stripped
# Default cost function is normalized correlation
# Why is the refvol 157?

mcflirt -in my_bold \
-out my_bold_mcf \
-cost normcorr \
-refvol 157 \
-mats -plots \
-rmsrel -rmsabs -spline_final 

Notes about mcflirt
* mats gives us a directory of .mat files, what do these represent?
* {out}.par is a text file with 6 columes, the first three columns are rotations for x,y,z and last three are translations for x,y,z
* Informative [old post](https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=fsl;cda6e2ea.1112) about the relation between the .MAT files and the .par parameters

In [ ]:
!cat my_bold_mcf.par

In [ ]:
%%bash

# FSL has a simple plotting command for timeseries data
fsl_tsplot

In [ ]:
%%bash

# Plot rotations (radians)
fsl_tsplot -i my_bold_mcf.par \
-t 'MCFLIRT estimated rotations (radians)' \
-u 1 --start=1 --finish=3 -a x,y,z -w 640 -h 144 -o rot.png 

Plot of rotations: 


In [ ]:
%%bash

# Plot translations (mm)
fsl_tsplot -i my_bold_mcf.par \
-t 'MCFLIRT estimated translations (mm)' \
-u 1 --start=4 --finish=6 -a x,y,z -w 640 -h 144 -o trans.png 

Plot of translations:

In [ ]:
%%bash

# Plot displacement. Abs=relative to reference vol. Rel=relative to vol n-1.
fsl_tsplot -i my_bold_mcf_abs.rms,my_bold_mcf_rel.rms \
-t 'MCFLIRT estimated mean displacement (mm)' \
-u 1 -w 640 -h 144 -a absolute,relative -o disp.png 

Plot of displacements:

### Question: 
* How can you tell what the reference volume is from looking at the figures of rotations, translations, and displacement above?
* What is the difference between the absolute and relative traces in the mean displacement plot?

Notes:

In [ ]:
%%bash

# Let's compare our functional volume from before to after motion correction in FSLview
fslview

In [ ]:
%%bash

# Did we get rid of ALL frame wise displacement?
# Run the fsl_motion_outliers program on the motion corrected data to get FD post mc
# Note you wouldn't typically do this for reporting FD, this is just for visualization


In [ ]:
# Python code to get mean FD post mc

Plot of FD post mc:

In [ ]:
%%bash

# How would we run fsl_motion_outliers to estimate dvars on data motion corrected once?

## Slice timing correction in FSL
* Slice timing is often done following motion correction
* The purpose is to adjust the bold signal in each slice to the best estimate of what it would have been if all the slices could have been acquired at the same time. 
* In order to do slice timing correction you'll typically need:
    * the order of slice acquisition 
    * the TR
    * with some programs you may be able to specify what the reference slice

In [ ]:
%%bash

# Usage
slicetimer

In [ ]:
%%bash

slicetimer -i my_bold_mcf.nii.gz \
-r 2 \
-o my_bold_mcf_st.nii.gz

In [ ]:
%%bash

# Look at the images, any apparent differences?
# Let's make a mask of the motor cortex and plot the timeseries from it to compare with and without st 
# Name the mask: my_bold_right_mot_mask.nii.gz
fslview

In [ ]:
%%bash

# Extract timeseries with fslmeants

# With mc only: mcf_right_mot_ts.txt


# With mc + st: mcf_st_right_mot_ts.txt


In [ ]:
mcf_mot = pd.read_table('mcf_right_mot_ts.txt', header=None)
mcf_st_mot = pd.read_table('mcf_st_right_mot_ts.txt', header=None)

In [ ]:
plt.plot(mcf_mot,label='mcf_mot') #plot the variable
plt.plot(mcf_st_mot,label='mcf_st_mot',alpha=.5)
plt.xlabel('Time (seconds)')
plt.ylabel('BOLD')
plt.legend(loc='best')

# Practice problems

### 1. What if we did slice timing correction first?

Use what you've learned above to run slice timing first followed by motion correction. It is not uncommon to see this done, so let's see what the effect is on the data by comparing the final estimate of FD with a plot.
* Name your slice-time corrected image my_bold_st.nii.gz
* Name your slice-time corrected + motion corrected image my_bold_st_mcf.nii.gz

In [ ]:
%%bash

# code to do slice-timing on the raw bold



In [ ]:
%%bash

# code to use fsl_motion_outliers to estimate FD on the slice-time corrected func image



In [ ]:
# python code to compare FD from the raw func with FD estimated on the slice-time corrected func

In [ ]:
%%bash

# code to do motion correction on st image with mcflirt


In [ ]:
%%bash

# Plot rotations (radians) of motion on slice-time corrected functional


In [ ]:
%%bash 

# Plot translations (mm) of motion on slice-time corrected functional


In [ ]:
%%bash 

# Plot displacement (mm) of motion on slice-time corrected functional

### 2. Observations

a) Was there any obvious difference on FD estimates when doing slice timing first? 


b) Based on the comparison and our discussions, which is better to run first, motion correction or slice timing?